In [40]:
import random
import copy

In [41]:
def next_piece():
    return random.choice(['R', 'G', 'B', 'Y']), random.choice(['R', 'G', 'B', 'Y'])

In [46]:
pieces = [next_piece() for _ in range(100)]
board = [[" " for _ in range(4)] for _ in range(9)]

In [47]:
board

[[' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ']]

# Puyo Env

In [105]:
class PuyoGameState:
    
    def __init__(self, board, pieces, turn=0):
        self.board_height = 9
        self.board_columns = 4
        self.board = board  # 2D list (height x width)
        self.pieces = pieces  # List of (color1, color2) tuples
        self.turn = turn  # Which piece we're on

    def get_moves(self):
        """ Required """
        """ Returns list of valid moves in ((column1, row1, color1), (column2, row2, color2)) format"""
        piece = self.pieces[self.turn]
        col1, col2 = piece
        moves = []

        # For simplicity, assume 4 rotations: 0 = up, 1 = right, 2 = down, 3 = left
        for rotation in range(4):
            for column in range(self.board_columns):
                pos1, pos2, valid = self.placement_position(self, column, rotation, board)
                if valid:
                    moves.append((pos1 + col1, pos2 + col2))
        return moves

    def placement_position(self, column, rotation, board):
        """Returns ((col1, row1), (col2, row2), valid_position)"""
        # 1: Check that we do not go out of the left/right bounds
        if column == 0:
            if rotation == 3:
                return (0, 0, False)
        elif column == self.board_columns - 1:
            if rotation == 1:
                return (0, 0, False)
            
        p1_height = self.get_column_height(column, board)
            
        # 2: Vertical placement
        if rotation == 0:
            if p1_height - 2 >= 0:
                return ((column, p1_height-1), (column, p1_height-2), True)
        elif rotation == 2:
            if p1_height - 2 >= 0:
                return ((column, p1_height-2), (column, p1_height-1), True)
        
        # 3: Horizontal placement
        else:
            if rotation == 1:
                p2_height = self.get_column_height(column + 1, board)
                if (p1_height - 1 >= 0 and p2_height - 1 >= 0):
                    return ((column, p1_height-1), (column+1, p2_height-1), True)
                else:
                    return (0, 0, False)
                
            elif rotation == 3:
                p2_height = self.get_column_height(column - 1, board)
                if (p1_height - 1 >= 0 and p2_height - 1 >= 0):
                    return ((column, p1_height-1), (column-1, p2_height-1), True)
                else:
                    return (0, 0, False)
            else:
                print("FUCK")
                return "FUCK"

    def get_column_height(self, column, board):
        for y in range(self.board_columns):
            if board[y][column] != 0:
                return y
        return self.board_height # empty column

    def do_move(self, move):
        """ Required """
        column, rotation = move
        new_board = copy.deepcopy(self.board)

        # Drop the current piece into the board
        # You'll need real logic here to update the board and resolve chains
        self.drop_piece(new_board, self.upcoming_pieces[self.turn], column, rotation)

        return PuyoGameState(new_board, self.upcoming_pieces, turn=self.turn + 1)

    def is_terminal(self):
        """ Required """
        # Top-out check or no more pieces
        return self.turn >= len(self.upcoming_pieces) or self.is_board_full()

    def get_result(self):
        """ Required """
        # Return a scalar value (e.g., chain length or heuristic)
        return self.evaluate_board()

    def place_piece(self, piece):
        (col1, row1, color1), (col2, row2, color2) = piece
        self.board[row1][col1] = color1
        self.board[row2][col2] = color2

    def remove_puyo(self, chained_puyos):
        for group in chained_puyos:
            for row, col in group:
                self.board[row][col] = " "

    def apply_gravity(self):
        for col in range(self.board_columns):
            queue = []
            for row in reversed(range(self.board_height)):
                # Collect non-empty cells from bottom to top
                if self.board[row][col] != " ":
                    queue.append(self.board[row][col])
            # Fill up the column again from the stack
            for row in reversed(range(self.board_height)):
                self.board[row][col] = queue.pop(0) if queue else " "

    def find_chained_puyos(self):
        """ Uses standard DFS algo to find chained puyos """
        """ Returns a list of lists, each individual list contains the cells of the group of chained puyos"""
        visited = [[False for _ in range(self.board_columns)] for _ in range(self.board_height)]
        chained_puyos = []

        for row in range(self.board_height):
            for col in range(self.board_columns):
                if self.board[row][col] != " " and not visited[row][col]:
                    color = self.board[row][col]
                    group = []
                    stack = [(row, col)]

                    while stack:
                        crow, ccol = stack.pop()
                        if not (0 <= crow < self.board_height and 0 <= ccol < self.board_columns):
                            continue
                        if visited[crow][ccol] or self.board[crow][ccol] != color:
                            continue

                        visited[crow][ccol] = True
                        group.append((crow, ccol))

                        for drow, dcol in [(-1,0), (1,0), (0,-1), (0,1)]:
                            stack.append((crow + drow, ccol + dcol))

                    if len(group) >= 4:
                        chained_puyos.append(group)

        return chained_puyos

    def evaluate_board(self):
        # Return a simple heuristic for now (e.g., count of cleared puyos)
        return random.random()  # Replace with actual evaluation

def print_board(board):
    emoji_map = {
        'R': '🟥',
        'G': '🟩',
        'B': '🟦',
        'Y': '🟨',
        'P': '🟪',
        ' ': '⬛'
    }

    for row in board:
        print('|' + ''.join(emoji_map.get(cell, '?!') for cell in row) + '|')
    print(" ")


# Modular Tests

In [106]:
example_board = [
    [' ', ' ', ' ', ' '],
    [' ', ' ', ' ', ' '],
    [' ', 'G', ' ', 'R'],
    [' ', 'G', ' ', 'R'],
    ['R', ' ', ' ', 'B'],
    ['R', ' ', ' ', 'B'],
    ['G', ' ', ' ', 'B'],
    ['G', 'Y', 'B', 'Y'],
    ['R', 'R', 'Y', 'Y'],
]

pieces = []  # No pieces needed for this test

# Instantiate game state
state = PuyoGameState(example_board, pieces)
print_board(state.board)
# Testing of functions
# find chains, should be empty
chained = state.find_chained_puyos()
print(f"Chained puyos: {chained}\n")

# Step 1: Apply gravity
state.apply_gravity()
print_board(state.board)

# Step 2: Find chains
chained = state.find_chained_puyos() # empty output
print(f"Chained puyos: {chained}")

# Step 3: Pop puyos
state.remove_puyo(chained)
print_board(state.board)

print("=================================")
# Loop until no chains
while chained:
    state.apply_gravity()
    print_board(state.board)
    chained = state.find_chained_puyos()
    print(f"Chained puyos: {chained}\n")
    state.remove_puyo(chained)
    print_board(state.board)




|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛🟩⬛🟥|
|⬛🟩⬛🟥|
|🟥⬛⬛🟦|
|🟥⬛⬛🟦|
|🟩⬛⬛🟦|
|🟩🟨🟦🟨|
|🟥🟥🟨🟨|
 
Chained puyos: []

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|🟥⬛⬛🟦|
|🟥🟩⬛🟦|
|🟩🟩⬛🟦|
|🟩🟨🟦🟨|
|🟥🟥🟨🟨|
 
Chained puyos: [[(5, 1), (6, 1), (6, 0), (7, 0)]]
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|🟥⬛⬛🟦|
|🟥⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛🟨🟦🟨|
|🟥🟥🟨🟨|
 
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|🟥⬛⬛🟦|
|🟥🟨🟦🟨|
|🟥🟥🟨🟨|
 
Chained puyos: [[(6, 0), (7, 0), (8, 0), (8, 1)]]

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛🟨🟦🟨|
|⬛⬛🟨🟨|
 
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛🟦🟨|
|⬛🟨🟨🟨|
 
Chained puyos: [[(7, 3), (8, 3), (8, 2), (8, 1)]]

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛🟦⬛|
|⬛⬛⬛⬛|
 
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛🟦🟦|
 
Chained puyos: [[(6, 3), (7, 3), (8, 3), (8, 2)]]

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
 
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
 
Chained puyos: []

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
 


In [50]:
def find_connected_groups(board):
    height = len(board)
    width = len(board[0])
    visited = [[False for _ in range(width)] for _ in range(height)]
    groups = []

    for y in range(height):
        for x in range(width):
            if board[y][x] != ' ' and not visited[y][x]:
                color = board[y][x]
                group = []
                stack = [(y, x)]

                while stack:
                    cy, cx = stack.pop()
                    if not (0 <= cy < height and 0 <= cx < width):
                        continue
                    if visited[cy][cx] or board[cy][cx] != color:
                        continue

                    visited[cy][cx] = True
                    group.append((cy, cx))

                    for dy, dx in [(-1,0), (1,0), (0,-1), (0,1)]:
                        stack.append((cy + dy, cx + dx))

                if len(group) >= 4:
                    groups.append(group)

    return groups

test_board = [
    # 0    1    2    3    4    5
    [' ', ' ', ' ', ' ', ' ', 'Y'], # 0
    [' ', ' ', ' ', ' ', ' ', 'Y'], # 1
    [' ', ' ', ' ', 'R', ' ', 'Y'], # 2
    [' ', ' ', 'B', 'R', 'R', 'B'], # 3
    [' ', 'G', 'B', 'R', 'R', 'B'], # 4
    ['R', 'G', 'B', 'B', 'Y', 'Y'], # 5
]


In [51]:
find_connected_groups(test_board)

[[(2, 3), (3, 3), (3, 4), (4, 4), (4, 3)], [(3, 2), (4, 2), (5, 2), (5, 3)]]

In [39]:
def apply_gravity(board):
    board_height = len(board)
    board_columns = len(board[0])
    
    for col in range(board_columns):
        queue = []
        for row in reversed(range(board_height)):
            if board[row][col] != " ":
                queue.append(board[row][col])
        for row in reversed(range(board_height)):
            board[row][col] = queue.pop() if queue else " "
    return board

# Example test board (rows x columns = 6 x 6)
test_board = [
    [' ', ' ', ' ', ' ', ' ', '$'],
    [' ', ' ', ' ', ' ', ' ', ' '],
    [' ', ' ', ' ', 'A', ' ', ' '],
    [' ', ' ', 'B', 'A', ' ', ' '],
    [' ', 'C', 'B', ' ', ' ', ' '],
    ['D', 'C', ' ', ' ', ' ', ' '],
]

# Apply gravity
result_board = apply_gravity(test_board)

result_board

[[' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' '],
 [' ', 'C', 'B', 'A', ' ', ' '],
 ['D', 'C', 'B', 'A', ' ', '$']]

To work with the mcts package, your game state class must implement:

Method	Description
get_moves()	Returns list of legal moves from this state
do_move(move)	Returns the new state after applying a move
is_terminal()	Returns True if game is over (e.g. top-out)
get_result()	Returns a numeric value or score for this state